In [156]:
import networkx as nx
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm

%matplotlib inline

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

n = 50
p = 0.885
N = 200

clique_small = np.ones((n,n)) - np.eye(n)
clique_large = np.ones((2*n,2*n)) - np.eye(2*n)

# Make schools
school_1 = nx.to_numpy_array(nx.erdos_renyi_graph(n, p))
school_2 = nx.to_numpy_array(nx.erdos_renyi_graph(n, p))
# Make zero block
zeros = np.zeros((n, n))
# Push blocks together
step1 = np.concatenate((school_1, zeros), axis=1)
step2 = np.concatenate((zeros.T, school_2), axis=1)

town = np.concatenate((step1, step2), axis=0)

Now that we have built our intial population, two schools with edges between schools randomly, we need to build the model for the evolving network dynamic.

In [157]:
a1 = 0.02
a2 = 0.02
w1 = 0.3
w2 = 0.5
eps = 0.016

# Make matrix A
A1 = a1 * clique_small
A2 = a2 * np.ones((n,n))

step1 = np.concatenate((A1, A2), axis=1)
step2 = np.concatenate((A2, A1), axis=1)
A = np.concatenate((step1, step2), axis=0)

# Make matrix Omega
W1 = w1 * clique_small
W2 = w2 * np.ones((n,n))

step1 = np.concatenate((W1, W2), axis=1)
step2 = np.concatenate((W2, W1), axis=1)
W = np.concatenate((step1, step2), axis=0)

# Make F(town) (expected graph at next step)
def next_exp_graph(curr):
    not_in = clique_large - curr
    Eps = eps * clique_large
    
    exp_born = not_in * (A + (Eps * np.matmul(curr, curr)))
    exp_survived = curr * (clique_large - W)
    
    return exp_born + exp_survived

# Make the time steps themselves
def one_time_step(curr):
    exp_next = next_exp_graph(curr)
    act_next = np.zeros((2*n,2*n))
    
    for i in range(2*n):
        for j in range(i+1, 2*n):
            rnd = np.random.choice([0, 1], p=[1-exp_next[i][j], exp_next[i][j]])
            act_next[i][j] = rnd
            act_next[j][i] = rnd
    
    return act_next

# Run simulation N times
def simulation(start, num_steps):
    states = []
    curr = start
    
    # Record a number of successive states
    for i in tqdm(range(num_steps)):
        states.append(curr)
        curr = one_time_step(curr)
    
    return states

# Function for the edge density of a network
def density(state):
    k = state.shape[0]    
    return (1 / (k * (k - 1))) * np.sum(state)

# Function to return intra- and inter-school edge density
def densities(state):
    count1 = 0
    count2 = 0
    for i in range(n):
        # Intra-school
        for j in range(i+1, n):
            count1 += state[i][j]
        # Inter-school    
        for j in range(n+1, 2*n):
            count2 += state[i][j]
            
    dens1 = (2 / (n * (n - 1))) * count1
    dens2 = (1 / (n * (n - 1))) * count2
    
    return dens1, dens2

# Run simulations
X = list(range(N))
colors = ['#FF7F50', '#CCCCFF']
simulations = simulation(town, N)
densities1 = [densities(state)[0] for state in simulations]
densities2 = [densities(state)[1] for state in simulations]

plt.plot(X, densities1, color=colors[0], label='Intra-school friendships')
plt.plot(X, densities2, color=colors[1], label='Inter-school friendships')
for i in range(7):
    simulations = simulation(town, N)
    densities1 = [densities(state)[0] for state in simulations]
    densities2 = [densities(state)[1] for state in simulations]

    plt.plot(X, densities1, color=colors[0])
    plt.plot(X, densities2, color=colors[1])

plt.xlabel('Time Steps')
plt.ylabel('Edge Density')
plt.legend(loc='upper left')
plt.savefig('edge_densities_pairs.pgf')

100%|█████████████████████████████████████████| 200/200 [00:17<00:00, 11.12it/s]
